<a href="https://colab.research.google.com/github/dnaae/spot_detection_/blob/cell-%26-nuclei-segmentation/Notebook_4_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install big-fish

In [ ]:
#imports
import os
import bigfish.stack as stack
import bigfish.segmentation as segmentation
import bigfish.plot as plot
import bigfish.multistack as multistack
from google.colab import drive

In [3]:
# Function to mount Google Drive and select a folder
def choose_folder_colab():
    drive.mount('/content/drive')
    drive_folder = "/content/drive/MyDrive/"
    contents = os.listdir(drive_folder)

    print("Contents of your Google Drive:")
    for i, item in enumerate(contents):
        print(f"{i + 1}: {item}")

    while True:
        choice = input("Enter the number of the folder you want to select (q to quit): ")
        if choice.lower() == 'q':
            return None
        try:
            choice = int(choice)
            if 1 <= choice <= len(contents):
                selected_folder = os.path.join(drive_folder, contents[choice - 1])
                return selected_folder
            else:
                print("Invalid choice. Please try again or enter 'q' to quit.")
        except (ValueError, KeyboardInterrupt):
            print("Invalid input. Please try again or enter 'q' to quit.")

In [16]:
def choose_image_index(path_output):
    path = os.path.join(path_output, "splitted images")
    splitted_images = [filename for filename in os.listdir(path) if filename.endswith(".tif")]

    print("Splitted images in the output folder:")
    for i, item in enumerate(splitted_images):
        print(f"{i + 1}: {item}")

    try:
        choice = input("Enter the index of the splitted image you want to process (q to quit): ")
        if choice.lower() == 'q':
            return None
        choice = int(choice)
        if 1 <= choice <= len(splitted_images):
            selected_image = splitted_images[choice - 1]
            return choice, selected_image  # Return the selected index and image name
        else:
            print("Invalid choice. Exiting.")
            return None
    except (ValueError, KeyboardInterrupt):
        if choice.lower() == 'q':
            return None
        print("Invalid input. Exiting.")
        return None

In [5]:
# Function to create folders for masks
def create_mask_folders(path_output):
    nuclear_mask_folder = os.path.join(path_output, "nuclear masks")
    cellular_mask_folder = os.path.join(path_output, "cellular masks")

    os.makedirs(nuclear_mask_folder, exist_ok=True)
    os.makedirs(cellular_mask_folder, exist_ok=True)

    return nuclear_mask_folder, cellular_mask_folder

In [29]:
def process_tiff_image(path_output, dapi_channel_name, smfish_channel_name, selected_image_index):
    # Construct the full path to the image file within the specified folder
    dapi_path = os.path.join(path_output, "splitted images", dapi_channel_name)
    smfish_path = os.path.join(path_output, "splitted images", smfish_channel_name)

    # Check if the image files exist before processing
    if not os.path.exists(dapi_path) or not os.path.exists(smfish_path):
        print(f"Image files not found in the specified folder.")
        return

    # Read DAPI and smFISH images
    dapi_3d = stack.read_image(dapi_path)
    dapi = stack.mean_projection(dapi_3d)
    smfish_3d = stack.read_image(smfish_path)
    smfish = stack.mean_projection(smfish_3d)

    # Process the images to obtain nuclear and cellular masks
    nuc_label, cell_label = extract_nuclear_mask_and_segment_cells(dapi, smfish)

    # Create folders for masks
    nuc_masks_folder, cell_masks_folder = create_mask_folders(path_output)

    # Define the names for the nuclear and cellular masks based on the selected index
    initial_image_index = selected_image_index + 1
    nuc_mask_name = f"nuclear_mask_{initial_image_index}.tif"
    cell_mask_name = f"cellular_mask_{initial_image_index}.tif"

    # Construct the full paths to save the masks in their respective folders
    nuc_mask_path = os.path.join(nuc_masks_folder, nuc_mask_name)
    cell_mask_path = os.path.join(cell_masks_folder, cell_mask_name)

    # Save the masks in the respective folders
    stack.save_image(nuc_label, nuc_mask_path)
    stack.save_image(cell_label, cell_mask_path)

    print(f"Processed image: {dapi_channel_name}")
    print(f"Saved nuclear mask as {nuc_mask_path}")
    print(f"Saved cellular mask as {cell_mask_path}")



In [30]:
# Function to extract nuclear mask and segment cells
def extract_nuclear_mask_and_segment_cells(dapi, smfish):
    # Your existing code for nuclear mask and cell segmentation
    nuc_mask = segmentation.thresholding(dapi, threshold=300)
    nuc_mask = segmentation.clean_segmentation(nuc_mask, small_object_size=100, fill_holes=True)
    nuc_label = segmentation.label_instances(nuc_mask)
    cell_label = segmentation.cell_watershed(smfish, nuc_label, threshold=60, alpha=0.9)
    nuc_label = segmentation.clean_segmentation(nuc_label, delimit_instance=True)
    cell_label = segmentation.clean_segmentation(cell_label, smoothness=7, delimit_instance=True)
    nuc_label, cell_label = multistack.match_nuc_cell(nuc_label, cell_label, single_nuc=False, cell_alone=True)
    return nuc_label, cell_label

if __name__ == "__main__":
    while True:
        path_input = choose_folder_colab()

        if path_input is None:
            break  # Exit the loop if 'q' is entered

        print(f"Selected folder: {path_input}")

        # Check if the selected folder has an "output" subfolder
        path_output = os.path.join(path_input, "output")

        if os.path.exists(path_output):
            # Change the working directory to the "output" subfolder
            os.chdir(path_output)

            # Allow the user to choose a subfolder interactively
            subfolders = [f for f in os.listdir() if os.path.isdir(f)]
            print("Subfolders:")
            for i, subfolder in enumerate(subfolders):
                print(f"{i + 1}: {subfolder}")

            selected_subfolder_index = int(input("Enter the number of the subfolder you want to select (0 for current folder): ")) -1

            if selected_subfolder_index >= 0 and selected_subfolder_index <= len(subfolders):
                selected_subfolder = subfolders[selected_subfolder_index]
                os.chdir(selected_subfolder)  # Change the working directory to the selected subfolder

                # List DAPI and FISH channels from the selected subfolder
                dapi_channels = [f for f in os.listdir() if "_C1.tif" in f]
                fish_channels = [f for f in os.listdir() if "_C2.tif" in f]

                print("Available DAPI channels:")
                for i, dapi_channel in enumerate(dapi_channels):
                    print(f"{i + 1}: {dapi_channel}")

                # Process the selected DAPI and FISH channels
                dapi_choice = int(input("Enter the number of the DAPI channel you want to select: ")) - 1
                selected_dapi_channel = dapi_channels[dapi_choice]

                print("Available FISH channels:")
                for i, fish_channel in enumerate(fish_channels):
                  print(f"{i + 1}: {fish_channel}")

                fish_choice = int(input("Enter the number of the FISH channel you want to select: ")) - 1
                selected_fish_channel = fish_channels[fish_choice]

                # Process the selected DAPI and FISH channels
                process_tiff_image(path_output, selected_dapi_channel, selected_fish_channel, dapi_choice)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of your Google Drive:
1: CV - D_Mitsea.pdf
2: Colab Notebooks
3: rf_final(1).ipynb
4: Random Forest 7_05.ipynb
5: Classroom
6: split channels.ijm
7: NOTEBOOK_6.cell extraction.ipynb
8: CPTH2
9: 5-AZA
10: DMSO
11: FULV
12: experiment_1_dapi_fov_1.tif
13: experiment_1_smfish_fov_1.tif
14: example_nuc_full.tif
15: example_cell_full.tif
Enter the number of the folder you want to select (q to quit): 8
Selected folder: /content/drive/MyDrive/CPTH2
Subfolders:
1: nuclear masks
2: cellular masks
3: splitted images
4: notebook 5 output
5: extraction_results_csv_notebook6
Enter the number of the subfolder you want to select (0 for current folder): 3
Available DAPI channels:
1: MCF7_AREG_CPTH2_501.tif_C1.tif
2: MCF7_AREG_CPTH2_505.tif_C1.tif
3: MCF7_AREG_CPTH2_509.tif_C1.tif
4: MCF7_AREG_CPTH2_506.tif_C1.tif
5: MCF7_AREG_CPTH2_504.tif_C1.tif
6: MCF7_AREG_CPTH2_